# StackOverflow Reading Big Dataset

In [1]:
import findspark
findspark.init()

# Load Python Libraries
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

import time
start_time = time.time()

#import matplotlib.pyplot as plt
#%matplotlib inline

#Set dataset path
data_path='/data/textdata/StackOverflow/SOTorrent_Dataset_2018-09-23/'

In [2]:
# Create Spark Session Environment
spark = SparkSession.builder\
       .master("local[*]")\
       .appName("StackOverflowReadDataset")\
       .getOrCreate()

# create the Spark Context
#sc = spark.sparkContext

#sqlContext = SQLContext(spark)

## Reading Data

### 1- Badges Database

In [3]:
# Read badges

badges_schema = StructType([
    StructField('Id', IntegerType(), True),
    StructField('UserId', IntegerType(), True),
    StructField('Name', StringType(), True),
    StructField('Date', TimestampType(), True),
    StructField('Class', ByteType(), True),
    StructField('TagBased', ByteType(), True)
])

badges = spark.read.format("csv").options(sep=',',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",header='true')\
         .load(data_path+"Badges.csv",schema=badges_schema)

#badges.dtypes
badges.show(5)

+-----+------+-------+--------------------+-----+--------+
|   Id|UserId|   Name|                Date|Class|TagBased|
+-----+------+-------+--------------------+-----+--------+
|82946|  3718|Teacher|2008-09-15 08:55:...|    3|       0|
|82947|   994|Teacher|2008-09-15 08:55:...|    3|       0|
|82949|  3893|Teacher|2008-09-15 08:55:...|    3|       0|
|82950|  4591|Teacher|2008-09-15 08:55:...|    3|       0|
|82951|  5196|Teacher|2008-09-15 08:55:...|    3|       0|
+-----+------+-------+--------------------+-----+--------+
only showing top 5 rows



### 2- Comments Database

In [4]:
# Read Comments

comments_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('PostId', IntegerType()),
    StructField('Score', IntegerType()),
    StructField('Text', StringType()),
    StructField('CreationDate', TimestampType()),
    StructField('UserDisplayName', StringType()),
    StructField('UserId', IntegerType())
])


comments = spark.read.format("csv").options(sep=',',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",header='true')\
           .load(data_path+"Comments.csv",schema=comments_schema)

#comments.dtypes
comments.show(5)
#print((comments.count(), len(comments.columns)))

+---+------+-----+--------------------+--------------------+---------------+------+
| Id|PostId|Score|                Text|        CreationDate|UserDisplayName|UserId|
+---+------+-----+--------------------+--------------------+---------------+------+
|  2| 35314|    8|Yeah, I didn't be...|2008-09-06 08:09:...|              3|  null|
|  4| 35195|    0|I don't see an ac...|2008-09-06 08:42:...|            380|  null|
|  9| 47239|    0|Jonathan: Wow!  T...|2008-09-06 12:26:...|           4550|  null|
| 10| 45651|    6|It will help if y...|2008-09-06 13:38:...|            242|  null|
| 12| 47428|    3|One of the things...|2008-09-06 13:51:...|           4642|  null|
+---+------+-----+--------------------+--------------------+---------------+------+
only showing top 5 rows



### 3- CommentUrl Database

In [5]:
## Read CommentUrl

commenturl_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('PostId', IntegerType()),
    StructField('CommentId', IntegerType()),
    StructField('LinkType', StringType()),
    StructField('LinkPosition', StringType()),
    StructField('LinkAnchor', StringType()),
    StructField('Protocol', StringType()),
    StructField('RootDomain', StringType()),
    StructField('CompleteDomain', StringType()),
    StructField('Path', StringType()),
    StructField('Query', StringType()),
    StructField('FragmentIdentifier', StringType()),
    StructField('Url', StringType()),
    StructField('FullMatch', StringType())
])

#Extrai dados de YouTube Links CSV para Dataframe Spark
commenturl = spark.read.format("csv").options(sep=',',quote='"',header='false')\
             .load(data_path+"CommentUrl.csv",schema=commenturl_schema)

#commenturl.dtypes
commenturl.show(5)

+---+------+---------+--------+------------+--------------------+--------+-----------------+--------------------+--------------------+---------------+------------------+--------------------+--------------------+
| Id|PostId|CommentId|LinkType|LinkPosition|          LinkAnchor|Protocol|       RootDomain|      CompleteDomain|                Path|          Query|FragmentIdentifier|                 Url|           FullMatch|
+---+------+---------+--------+------------+--------------------+--------+-----------------+--------------------+--------------------+---------------+------------------+--------------------+--------------------+
|  1| 47428|       12|BareLink|      Middle|http://i.love.pet...|    http|         pets.com|     i.love.pets.com|    search/cats+dogs|           null|              null|http://i.love.pet...|http://i.love.pet...|
|  2| 47428|       12|BareLink|         End|http://i.love.pet...|    http|         pets.com|     i.love.pets.com|  search/pug+puppies|           null|  

### 4- PostBlockDiff Database

In [6]:
## Read PostBlockDiff

postblockdiff_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('PostId', IntegerType()),
    StructField('PostHistoryId', IntegerType()),
    StructField('LocalId', IntegerType()),
    StructField('PostBlockVersionId', IntegerType()),
    StructField('PredPostHistoryId', IntegerType()),
    StructField('PredLocalId', IntegerType()),
    StructField('PredPostBlockVersionId', IntegerType()),
    StructField('PostBlockDiffOperationId', IntegerType()),    
    StructField('Text', StringType()),
])

#Extrai dados de YouTube Links CSV para Dataframe Spark
postblockdiff = spark.read.format("csv").options(sep=',',header='false')\
                .load(data_path+"PostBlockDiff.csv",schema=postblockdiff_schema)

#postblockdiff.dtypes
postblockdiff.show(5)

+---+--------+-------------+-------+------------------+-----------------+-----------+----------------------+------------------------+--------------------+
| Id|  PostId|PostHistoryId|LocalId|PostBlockVersionId|PredPostHistoryId|PredLocalId|PredPostBlockVersionId|PostBlockDiffOperationId|                Text|
+---+--------+-------------+-------+------------------+-----------------+-----------+----------------------+------------------------+--------------------+
|  1|10139973|     23443092|      1|               105|         23242237|          1|                    96|                      -1|"How can I ""echo...|
|  2|10139973|     23443092|      1|               105|         23242237|          1|                    96|                       1|"anyone knows how...|
|  3|10139973|     23443092|      1|               105|         23242237|          1|                    96|                       0|          The query:|
|  4|10139973|     23443092|      2|               106|         232422

### 5- PostBlockVersion Database

In [7]:
## read PostBlockVersion.csv.gz
## 

postblockversion_schema = StructType([
    StructField('Id', IntegerType(), True),
    StructField('PostBlockTypeId', ByteType(), True),
    StructField('PostId', IntegerType(), True),
    StructField('PostHistoryId', IntegerType(), True),
    StructField('LocalId', IntegerType(), True),
    StructField('PredPostBlockVersionId', IntegerType(), True),
    StructField('PredPostHistoryId', IntegerType(), True),    
    StructField('PredLocalId', IntegerType(), True),    
    StructField('RootPostBlockVersionId', IntegerType(), True),    
    StructField('RootPostHistoryId', IntegerType(), True),    
    StructField('RootLocalId', IntegerType(), True),    
    StructField('PredEqual', BooleanType(), True),    
    StructField('PredSimilarity', DoubleType(), True),    
    StructField('PredCount', IntegerType(), True),    
    StructField('SuccCount', IntegerType(), True),    
    StructField('Length', IntegerType(), True),    
    StructField('LineCount', IntegerType(), True),    
    StructField('Content', StringType(), True)
])

#Read CSV to Dataframe
postblockversion = spark.read.format("csv").options(sep=',',quote='"',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",header='false')\
                   .load(data_path+"PostBlockVersion.csv",schema=postblockversion_schema)

#postblockversion.dtypes
postblockversion.show(5)

+----+---------------+--------+-------------+-------+----------------------+-----------------+-----------+----------------------+-----------------+-----------+---------+--------------+---------+---------+------+---------+--------------------+
|  Id|PostBlockTypeId|  PostId|PostHistoryId|LocalId|PredPostBlockVersionId|PredPostHistoryId|PredLocalId|RootPostBlockVersionId|RootPostHistoryId|RootLocalId|PredEqual|PredSimilarity|PredCount|SuccCount|Length|LineCount|             Content|
+----+---------------+--------+-------------+-------+----------------------+-----------------+-----------+----------------------+-----------------+-----------+---------+--------------+---------+---------+------+---------+--------------------+
|   1|              1|       4|      1049748|      1|                  null|             null|       null|                     1|          1049748|          1|     null|          null|        0|        1|    81|        2|I'm new to C#, an...|
|   2|              2|      

### 6- PostHistory Database

In [8]:
# Read PostHistory.xml.gz

posthistory_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('PostHistoryTypeId', ByteType()),
    StructField('PostId', IntegerType()),
    StructField('RevisionGUID', StringType()),
    StructField('CreationDate', TimestampType()),
    StructField('UserId', IntegerType()),
    StructField('UserDisplayName', StringType()),
    StructField('Comment', StringType()),
    StructField('Text', StringType())
])


#Read CSV to Dataframe
#posthistory = spark.read.format("csv").options(sep=',',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",header='true')\
#            .load(data_path+"PostHistory.csv",schema=posthistory_schema)


#posthistory = spark.read \
#              .format("com.databricks.spark.xml") \
#              .option("rowTag", "row") \
#              .options(samplingRatio=0.1) \
#              .load(data_path+"PostHistory.xml.gz",schema=posthistory_schema)

#posthistory.dtypes
#posthistory.show(5)

### 7- PostLinks Database

In [9]:
# Read PostLinks.xml.gz

postlinks_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('CreationDate', TimestampType()),
    StructField('PostId', IntegerType()),
    StructField('RelatedPostId', IntegerType()),
    StructField('LinkTypeId', ByteType())
])

#Read CSV to Dataframe
postlinks = spark.read.format("csv").options(sep=',',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",header='true')\
            .load(data_path+"PostLinks.csv",schema=postlinks_schema)

#postlinks.dtypes
postlinks.show(5)
#postlinks.schema

+---+--------------------+------+-------------+----------+
| Id|        CreationDate|PostId|RelatedPostId|LinkTypeId|
+---+--------------------+------+-------------+----------+
| 19|2010-04-26 02:59:...|   109|        32412|         1|
| 37|2010-04-26 02:59:...|  1970|       617600|         1|
| 42|2010-04-26 02:59:...|  2154|      2451138|         1|
| 48|2010-04-26 02:59:...|  2483|       496096|         1|
| 52|2010-04-26 02:59:...|  2572|       209329|         1|
+---+--------------------+------+-------------+----------+
only showing top 5 rows



### 8- PostReferenceGH Database

In [10]:
## Read PostReferenceGH

postreferencegh_schema = StructType([
    StructField('FileId', StringType()),
    StructField('RepoName', StringType()),
    StructField('Branch', StringType()),
    StructField('Path', StringType()),
    StructField('FileExt', StringType()),
    StructField('Size', IntegerType()),
    StructField('Copies', IntegerType()),
    StructField('PostId', IntegerType()),
    StructField('PostTypeId', ByteType()),
    StructField('SOUrl', StringType()),
    StructField('GHUrl', StringType())
])

#Extrai dados  CSV para Dataframe Spark
postreferencegh = spark.read.format("csv").options(sep=',',header='true')\
                 .load(data_path+"PostReferenceGH.csv",schema=postreferencegh_schema)

#postreferencegh.dtypes
postreferencegh.show(5)

+--------------------+--------------------+------+--------------------+-------+-----+------+--------+----------+--------------------+--------------------+
|              FileId|            RepoName|Branch|                Path|FileExt| Size|Copies|  PostId|PostTypeId|               SOUrl|               GHUrl|
+--------------------+--------------------+------+--------------------+-------+-----+------+--------+----------+--------------------+--------------------+
|227a8be680dfab54c...|at15/hadoop-spark...|master|provision/cluster...|    .sh|  537|     2| 4774054|         1|http://stackoverf...|https://raw.githu...|
|8cc491d8c007307b2...| gamepopper/MAN-Haxe|master|      sample/Main.hx|    .hx| 3967|     1|26877634|         1|http://stackoverf...|https://raw.githu...|
|fb21b030a68e92cec...|goaty92/Definitel...|master|angularjs/legacy/...|    .ts|53896|   332|17201854|         1|http://stackoverf...|https://raw.githu...|
|c0b9853033d70e493...|    apache/zookeeper|master|src/java/test/bin...

### 9- Posts Database

In [11]:
#Read Posts.xml.gz

posts_schema = StructType([
    StructField('Id', IntegerType(), True),
    StructField('PostTypeId', ByteType(), True),
    StructField('AcceptedAnswerId', IntegerType(), True),
    StructField('ParentId', IntegerType(), True),
    StructField('CreationDate', TimestampType(), True),
    StructField('Score', IntegerType(), True),
    StructField('ViewCount', IntegerType(), True),
    StructField('Body', StringType(), True),
    StructField('OwnerUserId', IntegerType(), True),
    StructField('OwnerDisplayName', StringType(), True),
    StructField('LastEditorUserId', IntegerType(), True),
    StructField('LastEditorDisplayName', StringType(), True),
    StructField('LastEditDate', TimestampType(), True),
    StructField('LastActivityDate', TimestampType(), True),
    StructField('Title', StringType(), True),
    StructField('Tags', StringType(), True),
    StructField('AnswerCount', IntegerType(), True),
    StructField('CommentCount', IntegerType(), True),
    StructField('FavoriteCount', IntegerType(), True)
])

#Read CSV to Dataframe
posts = spark.read.format("csv").options(sep=',',quote='"',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",escape='"',header='true')\
        .load(data_path+"Posts.csv",schema=posts_schema)


#    StructField('DeletionDate', TimestampType()),
#    StructField('ClosedDate', TimestampType()),
#    StructField('CommunityOwnedDate', TimestampType())

#posts.dtypes
posts.show(5)

+----+----------+----------------+--------+--------------------+-----+---------+--------------------+-----------+----------------+----------------+---------------------+------------+----------------+-----+----+-----------+------------+-------------+
|  Id|PostTypeId|AcceptedAnswerId|ParentId|        CreationDate|Score|ViewCount|                Body|OwnerUserId|OwnerDisplayName|LastEditorUserId|LastEditorDisplayName|LastEditDate|LastActivityDate|Title|Tags|AnswerCount|CommentCount|FavoriteCount|
+----+----------+----------------+--------+--------------------+-----+---------+--------------------+-----------+----------------+----------------+---------------------+------------+----------------+-----+----+-----------+------------+-------------+
|   4|         1|               7|    null|2008-07-31 21:42:...|  573|    37080|<p>I want to use ...|       null|            null|            null|                 null|        null|            null| null|null|       null|        null|         null|


### 10- PostVersion Database

In [12]:
## Read PostVersion

postversion_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('PostId', IntegerType()),
    StructField('PostTypeId', ByteType()),
    StructField('PostHistoryId', IntegerType()),
    StructField('PostHistoryTypeId', ByteType()),
    StructField('CreationDate', TimestampType()),
    StructField('PredPostHistoryId', IntegerType()),
    StructField('SuccPostHistoryId', IntegerType())
])

#Extrai dados de CSV para Dataframe Spark
postversion = spark.read.format("csv").options(sep=',',header='false',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS")\
              .load(data_path+"PostVersion.csv",schema=postversion_schema)

#postversion.schema
#postversion.dtypes
postversion.show(5)

+---+--------+----------+-------------+-----------------+-------------------+-----------------+-----------------+
| Id|  PostId|PostTypeId|PostHistoryId|PostHistoryTypeId|       CreationDate|PredPostHistoryId|SuccPostHistoryId|
+---+--------+----------+-------------+-----------------+-------------------+-----------------+-----------------+
|  1|44655755|         1|    149795765|                2|2017-06-20 14:24:16|             null|             null|
|  2|10139969|         1|     23242227|                2|2012-04-13 11:29:26|             null|             null|
|  3|27827982|         1|     81132051|                2|2015-01-07 20:22:41|             null|             null|
|  4|44655757|         1|    149795776|                2|2017-06-20 14:24:25|             null|             null|
|  5|10139970|         1|     23242230|                2|2012-04-13 11:29:27|             null|             null|
+---+--------+----------+-------------+-----------------+-------------------+-----------

### 11- PostVersionUrl Database

In [13]:
## Read PostVersionUrl

postversionurl_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('PostId', IntegerType()),
    StructField('PostHistoryId', IntegerType()),
    StructField('PostBlockVersionId', IntegerType()),
    StructField('LinkType', StringType()),
    StructField('LinkPosition', StringType()),
    StructField('LinkAnchor', StringType()),
    StructField('Protocol', StringType()),
    StructField('RootDomain', StringType()),
    StructField('CompleteDomain', StringType()),
    StructField('Path', StringType()),
    StructField('Query', StringType()),
    StructField('FragmentIdentifier', StringType()),
    StructField('Url', StringType()),
    StructField('FullMatch', StringType())
])

    
#    StructField('PostTypeId', IntegerType()),
#    StructField('PostHistoryTypeId', IntegerType()),
#    StructField('CreationDate', TimestampType()),
#    StructField('PredPostHistoryId', IntegerType()),
#    StructField('SuccPostHistoryId', IntegerType())


#Extrai dados CSV para Dataframe Spark
postversionurl = spark.read.format("csv").options(sep=',',header='false')\
                 .load(data_path+"PostVersionUrl.csv",schema=postversionurl_schema)

#postversionurl.dtypes
postversionurl.show(5)

+---+--------+-------------+------------------+--------------------+------------+--------------------+--------+-------------+--------------------+--------------------+-----+------------------+--------------------+--------------------+
| Id|  PostId|PostHistoryId|PostBlockVersionId|            LinkType|LinkPosition|          LinkAnchor|Protocol|   RootDomain|      CompleteDomain|                Path|Query|FragmentIdentifier|                 Url|           FullMatch|
+---+--------+-------------+------------------+--------------------+------------+--------------------+--------+-------------+--------------------+--------------------+-----+------------------+--------------------+--------------------+
|  1|44655755|    149795765|                21|MarkdownLinkRefer...|         End|[![enter image de...|   https|    imgur.com|   i.stack.imgur.com|           kjzj6.png| null|              null|https://i.stack.i...|[![enter image de...|
|  2|44655755|    149795765|                21|MarkdownLinkR

### 12- Tags Database

In [14]:
# Read Tags.csv

tags_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('TagName', StringType()),
    StructField('Count', IntegerType()),
    StructField('ExcerptPostId', IntegerType()),
    StructField('WikiPostId', IntegerType())
])

#Extrai dados CSV para Dataframe Spark
tags = spark.read.format("csv").options(sep=',',header='true')\
       .load(data_path+"Tags.csv",schema=tags_schema)

#tags.dtypes
tags.show(5)

+---+----------+-------+-------------+----------+
| Id|   TagName|  Count|ExcerptPostId|WikiPostId|
+---+----------+-------+-------------+----------+
|  1|      .net| 272947|      3624959|   3607476|
|  2|      html| 771613|      3673183|   3673182|
|  3|javascript|1675041|      3624960|   3607052|
|  4|       css| 551318|      3644670|   3644669|
|  5|       php|1223812|      3624936|   3607050|
+---+----------+-------+-------------+----------+
only showing top 5 rows



### 13- TitleVersion Database

In [15]:
## Read TitleVersion

titleversion_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('PostId', IntegerType()),
    StructField('PostTypeId', ByteType()),
    StructField('PostHistoryId', IntegerType()),
    StructField('PostHistoryTypeId', ByteType()),
    StructField('CreationDate', TimestampType()),
    StructField('Title', StringType()),
    StructField('PredPostHistoryId', IntegerType()),
    StructField('PredEditDistance', IntegerType()),
    StructField('SuccPostHistoryId', IntegerType()),
    StructField('SuccEditDistance', IntegerType())
])

#Extrai dados CSV para Dataframe Spark
titleversion = spark.read.format("csv").options(sep=',',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",header='false')\
               .load(data_path+"TitleVersion.csv",schema=titleversion_schema)

#titleversion.schema
#titleversion.dtypes
titleversion.show(5)

+---+--------+----------+-------------+-----------------+-------------------+--------------------+-----------------+----------------+-----------------+----------------+
| Id|  PostId|PostTypeId|PostHistoryId|PostHistoryTypeId|       CreationDate|               Title|PredPostHistoryId|PredEditDistance|SuccPostHistoryId|SuccEditDistance|
+---+--------+----------+-------------+-----------------+-------------------+--------------------+-----------------+----------------+-----------------+----------------+
|  1|44655755|         1|    149795766|                1|2017-06-20 14:24:16|Ionic 2 applicati...|             null|            null|             null|            null|
|  2|10139969|         1|     23242228|                1|2012-04-13 11:29:26|Connecting to SQL...|             null|            null|             null|            null|
|  3|27827982|         1|     81132052|                1|2015-01-07 20:22:41|how to dynamicall...|             null|            null|             null|    

### 14- Users Database

In [16]:
# Read Users.xml.gz

users_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('Reputation', IntegerType()),
    StructField('CreationDate', TimestampType()),
    StructField('DisplayName', StringType()),
    StructField('LastAccessDate', TimestampType()),
    StructField('WebsiteUrl', StringType()),
    StructField('Location', StringType()),
    StructField('ProfileImageUrl', StringType()),
    StructField('AboutMe', StringType()),
    StructField('Views', IntegerType()),
    StructField('UpVotes', IntegerType()),
    StructField('DownVotes', IntegerType()),
    StructField('Age', IntegerType()),
    StructField('AccountId', IntegerType()),
    StructField('EmailHash', StringType())
])

#Extrai dados CSV para Dataframe Spark
users = spark.read.format("csv").options(sep=',',quote='"',escape='"',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",header='true')\
        .load(data_path+"Users.csv",schema=users_schema)

#users.dtypes
users.show(5)

+----+----------+-------------------+-----------+--------------------+--------------------+------------------+---------------+--------------------+-----+-------+---------+----+---------+---------+
|  Id|Reputation|       CreationDate|DisplayName|      LastAccessDate|          WebsiteUrl|          Location|ProfileImageUrl|             AboutMe|Views|UpVotes|DownVotes| Age|AccountId|EmailHash|
+----+----------+-------------------+-----------+--------------------+--------------------+------------------+---------------+--------------------+-----+-------+---------+----+---------+---------+
|  -1|         1|2008-07-31 00:00:00|  Community|2008-08-26 00:16:...|http://meta.stack...|on the server farm|           null|<p>Hi, I'm not re...| null|   null|     null|null|     null|     null|
|null|      null|               null|       null|                null|                null|              null|           null|                null| null|   null|     null|null|     null|     null|
|null|      nul

### 15- Votes Database

In [17]:
# Read Votes.xml.gz

votes_schema = StructType([
    StructField('Id', IntegerType()),
    StructField('PostId', IntegerType()),
    StructField('VoteTypeId', ByteType()),
    StructField('CreationDate', TimestampType())
])

#    StructField('UserId', IntegerType()),
#    StructField('BountyAmount', IntegerType())

#Extrai dados CSV para Dataframe Spark
votes = spark.read.format("csv").options(sep=',',dateFormat="yyyy-MM-dd'T'HH:mm:ss.SSS",header='true')\
        .load(data_path+"Votes.csv",schema=votes_schema)

#votes.dtypes
votes.show(5)

+---+------+----------+-------------------+
| Id|PostId|VoteTypeId|       CreationDate|
+---+------+----------+-------------------+
|  1|     1|         2|2008-07-31 00:00:00|
|  2|     3|         2|2008-07-31 00:00:00|
|  3|     2|         2|2008-07-31 00:00:00|
|  4|     4|         2|2008-07-31 00:00:00|
|  5|     6|         2|2008-07-31 00:00:00|
+---+------+----------+-------------------+
only showing top 5 rows



## Bigdata Minning

In [ ]:
#Start code here.....

## Finishing Bigdata Evaluation

In [18]:
spark.stop()
print("--- Execution time: %s seconds ---" % (time.time() - start_time))

--- Execution time: 182.19748616218567 seconds ---
